# SE(3) Geometric Tracking Controller

This notebook explores the `controller_se3.py` module, which implements the geometric tracking controller from:

> Lee, T., Leok, M., & McClamroch, N. H. (2010). "Geometric Tracking Control of a Quadrotor UAV on SE(3)"

## Overview

The SE(3) controller works in two cascaded stages:

1. **Position Control**: Computes desired thrust direction and magnitude to track position/velocity
2. **Attitude Control**: Computes body moments to align with desired orientation

## Key Concepts

- **SE(3)**: The Special Euclidean group in 3D, representing rigid body poses (rotation + translation)
- **SO(3)**: The Special Orthogonal group in 3D, representing pure rotations
- **Geometric Control**: Control designed directly on the manifold, avoiding singularities of Euler angles

In [ ]:
# Setup: Add the src directory to Python path
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Import quadrotor modules
from quad.controller_se3 import (
    compute_desired_rotation,
    attitude_error,
    angular_rate_error,
    se3_controller,
    ControllerState,
)
from quad.types import State, Control, TrajPoint
from quad.params import Params
from quad.math3d import quat_to_R, R_to_quat, quat_to_euler, hat, vee
from quad.dynamics import step_rk4

# Set up plotting style
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

print("Modules loaded successfully!")

## 1. Desired Rotation Computation

Given a commanded acceleration $\mathbf{a}_{cmd}$ and desired yaw angle $\psi$, we construct the desired rotation matrix $R_{des}$:

1. **Body z-axis** ($\mathbf{b}_3$): Points along the thrust direction (normalized $\mathbf{a}_{cmd}$)
2. **Body x-axis** ($\mathbf{b}_1$): Derived from yaw while remaining orthogonal to $\mathbf{b}_3$
3. **Body y-axis** ($\mathbf{b}_2$): Completes the right-handed frame

$$R_{des} = [\mathbf{b}_1 | \mathbf{b}_2 | \mathbf{b}_3]$$

In [ ]:
# Example 1: Vertical thrust with zero yaw (hover)
a_cmd_hover = np.array([0.0, 0.0, 9.81])  # Pointing up
yaw_hover = 0.0

R_des_hover = compute_desired_rotation(a_cmd_hover, yaw_hover)

print("=== Hover Configuration ===")
print(f"Commanded acceleration: {a_cmd_hover}")
print(f"Yaw: {np.degrees(yaw_hover):.1f}°")
print(f"\nDesired rotation matrix:")
print(R_des_hover.round(4))
print(f"\nThis is close to identity (level attitude, no yaw)")

In [ ]:
# Example 2: Tilted thrust for lateral acceleration
# To accelerate forward (+x), we need to tilt the quad
# Commanded acceleration includes gravity compensation and lateral component
a_cmd_forward = np.array([3.0, 0.0, 9.81])  # Forward + up
yaw_forward = 0.0

R_des_forward = compute_desired_rotation(a_cmd_forward, yaw_forward)

print("=== Forward Acceleration ===")
print(f"Commanded acceleration: {a_cmd_forward}")
print(f"\nDesired rotation matrix:")
print(R_des_forward.round(4))

# Extract pitch angle from rotation matrix
# b3 (body z) is the 3rd column
b3 = R_des_forward[:, 2]
pitch = np.arcsin(-b3[0])  # pitch = arcsin(-R[0,2]) for small angles
print(f"\nBody z-axis (thrust direction): {b3.round(4)}")
print(f"Pitch angle: {np.degrees(pitch):.1f}° (tilted forward)")

In [ ]:
# Example 3: Effect of yaw on desired rotation
a_cmd = np.array([0.0, 0.0, 9.81])  # Pure vertical
yaw_angles = [0, 45, 90, 180]  # degrees

print("=== Yaw Effect on Rotation ===")
print("Body x-axis direction for different yaw angles:\n")

for yaw_deg in yaw_angles:
    yaw_rad = np.radians(yaw_deg)
    R_des = compute_desired_rotation(a_cmd, yaw_rad)
    b1 = R_des[:, 0]  # Body x-axis
    print(f"Yaw = {yaw_deg:3d}°: b1 = [{b1[0]:6.3f}, {b1[1]:6.3f}, {b1[2]:6.3f}]")

print("\nThe body x-axis rotates in the horizontal plane with yaw.")

In [ ]:
# Visualize desired rotation for different accelerations
fig = plt.figure(figsize=(14, 5))

test_cases = [
    (np.array([0, 0, 10]), 0, "Hover"),
    (np.array([5, 0, 10]), 0, "Forward accel"),
    (np.array([0, 5, 10]), 0, "Right accel"),
    (np.array([0, 0, 10]), np.pi/4, "Yaw 45°"),
]

for i, (a_cmd, yaw, title) in enumerate(test_cases):
    ax = fig.add_subplot(1, 4, i+1, projection='3d')
    
    R_des = compute_desired_rotation(a_cmd, yaw)
    
    # Draw body axes
    origin = np.zeros(3)
    colors = ['r', 'g', 'b']  # x=red, y=green, z=blue
    labels = ['x', 'y', 'z']
    
    for j in range(3):
        axis = R_des[:, j]
        ax.quiver(*origin, *axis, color=colors[j], arrow_length_ratio=0.1,
                  linewidth=2, label=f'b{j+1} ({labels[j]})')
    
    # Draw commanded acceleration (normalized)
    a_norm = a_cmd / np.linalg.norm(a_cmd)
    ax.quiver(*origin, *a_norm, color='orange', arrow_length_ratio=0.1,
              linewidth=2, linestyle='--', label='a_cmd')
    
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(title)
    if i == 0:
        ax.legend(loc='upper left', fontsize=8)

plt.tight_layout()
plt.show()

## 2. Attitude Error on SO(3)

The attitude error is defined geometrically on SO(3):

$$e_R = \frac{1}{2} \text{vee}(R_{des}^T R - R^T R_{des})$$

where $\text{vee}(\cdot)$ extracts a vector from a skew-symmetric matrix.

Properties:
- $e_R = 0$ when $R = R_{des}$
- Globally defined (no singularities unlike Euler angles)
- Has nice stability properties for control design

In [ ]:
# Test attitude error at equilibrium
R_current = np.eye(3)
R_desired = np.eye(3)

e_R = attitude_error(R_current, R_desired)

print("=== Attitude Error at Equilibrium ===")
print(f"Current R = Desired R = Identity")
print(f"Attitude error: {e_R}")
print(f"Error norm: {np.linalg.norm(e_R):.2e}")
print("\nAs expected, error is zero when aligned.")

In [ ]:
# Test attitude error for various rotations
def rotation_x(theta):
    """Rotation matrix about x-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])

def rotation_y(theta):
    """Rotation matrix about y-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])

def rotation_z(theta):
    """Rotation matrix about z-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

print("=== Attitude Error for Various Misalignments ===")
R_des = np.eye(3)

test_rotations = [
    (rotation_x(np.radians(10)), "10° roll"),
    (rotation_y(np.radians(10)), "10° pitch"),
    (rotation_z(np.radians(10)), "10° yaw"),
    (rotation_x(np.radians(30)), "30° roll"),
    (rotation_y(np.radians(45)), "45° pitch"),
]

for R_current, name in test_rotations:
    e_R = attitude_error(R_current, R_des)
    print(f"{name:12s}: e_R = [{e_R[0]:7.4f}, {e_R[1]:7.4f}, {e_R[2]:7.4f}], |e_R| = {np.linalg.norm(e_R):.4f}")

In [ ]:
# Visualize attitude error as function of angle
angles = np.linspace(0, np.pi, 100)
error_norms = []

R_des = np.eye(3)
for theta in angles:
    R_current = rotation_x(theta)  # Rotate about x-axis
    e_R = attitude_error(R_current, R_des)
    error_norms.append(np.linalg.norm(e_R))

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(np.degrees(angles), error_norms, 'b-', linewidth=2)
plt.xlabel('Rotation Angle [deg]')
plt.ylabel('||e_R||')
plt.title('Attitude Error Magnitude vs Rotation Angle')
plt.grid(True)

# Show that error is approximately linear for small angles
plt.subplot(1, 2, 2)
small_angles = angles[angles < np.radians(30)]
small_errors = error_norms[:len(small_angles)]
plt.plot(small_angles, small_errors, 'b-', linewidth=2, label='Actual')
plt.plot(small_angles, small_angles, 'r--', linewidth=1, label='θ (linear)')
plt.xlabel('Rotation Angle [rad]')
plt.ylabel('||e_R||')
plt.title('Small Angle Regime (Linear Approximation)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print("For small angles, ||e_R|| ≈ θ (attitude error equals rotation angle)")

## 3. Angular Rate Error

The angular rate error accounts for the transformation between frames:

$$e_\omega = \omega - R^T R_{des} \omega_{des}$$

This transforms the desired angular velocity into the current body frame before computing the difference.

In [ ]:
# Angular rate error examples
R = np.eye(3)
R_des = np.eye(3)

# Case 1: Both at rest
w = np.zeros(3)
w_des = np.zeros(3)
e_w = angular_rate_error(w, w_des, R, R_des)
print("=== Angular Rate Error ===")
print(f"Case 1: Both at rest")
print(f"  e_ω = {e_w}")

# Case 2: Current rotating, desired at rest
w = np.array([0.5, 0.0, 0.0])  # Rolling
w_des = np.zeros(3)
e_w = angular_rate_error(w, w_des, R, R_des)
print(f"\nCase 2: Current rolling at 0.5 rad/s, desired at rest")
print(f"  e_ω = {e_w}")

# Case 3: Desired yaw rate
w = np.zeros(3)
w_des = np.array([0.0, 0.0, 0.3])  # Desired yaw rate
e_w = angular_rate_error(w, w_des, R, R_des)
print(f"\nCase 3: Desired yaw rate of 0.3 rad/s, currently at rest")
print(f"  e_ω = {e_w}")

## 4. Full SE(3) Controller

The complete controller computes:

**Position Control:**
$$\mathbf{a}_{cmd} = \mathbf{a}_{des} - K_p (\mathbf{p} - \mathbf{p}_{des}) - K_d (\mathbf{v} - \mathbf{v}_{des}) + g \mathbf{e}_3$$

$$T = m \cdot \mathbf{a}_{cmd} \cdot (R \mathbf{e}_3)$$

**Attitude Control:**
$$\boldsymbol{\tau} = -K_R e_R - K_\omega e_\omega + \boldsymbol{\omega} \times (J \boldsymbol{\omega})$$

The gyroscopic term $\omega \times (J\omega)$ provides feedforward compensation.

In [ ]:
# Test the controller at hover equilibrium
params = Params()

# State: hovering at origin
state = State.zeros()
state.p = np.array([0.0, 0.0, 1.0])

# Trajectory: hover at the same position
traj = TrajPoint.hover(position=np.array([0.0, 0.0, 1.0]))

# Run controller
control, ctrl_state = se3_controller(state, traj, params)

print("=== Hover Equilibrium ===")
print(f"Position: {state.p}")
print(f"Desired position: {traj.p}")
print(f"\nControl outputs:")
print(f"  Thrust: {control.thrust_N:.3f} N (hover = {params.hover_thrust:.3f} N)")
print(f"  Moments: {control.moments_Nm.round(6)}")
print(f"\nController errors:")
print(f"  Position error: {ctrl_state.e_pos}")
print(f"  Velocity error: {ctrl_state.e_vel}")
print(f"  Attitude error: {ctrl_state.e_att}")
print(f"  Rate error: {ctrl_state.e_rate}")

In [ ]:
# Test with position error
state = State.zeros()
state.p = np.array([0.0, 0.0, 0.5])  # Below target

traj = TrajPoint.hover(position=np.array([0.0, 0.0, 1.0]))

control, ctrl_state = se3_controller(state, traj, params)

print("=== Position Error (Below Target) ===")
print(f"Position: {state.p}")
print(f"Desired: {traj.p}")
print(f"Position error: {ctrl_state.e_pos}")
print(f"\nControl outputs:")
print(f"  Thrust: {control.thrust_N:.3f} N (> hover = {params.hover_thrust:.3f} N)")
print(f"\nThe controller commands more thrust to climb up.")

In [ ]:
# Test with lateral position error
state = State.zeros()
state.p = np.array([0.0, 0.0, 1.0])  # At hover altitude

traj = TrajPoint.hover(position=np.array([1.0, 0.0, 1.0]))  # Target is +X

control, ctrl_state = se3_controller(state, traj, params)

print("=== Lateral Position Error ===")
print(f"Position: {state.p}")
print(f"Desired: {traj.p}")
print(f"Position error: {ctrl_state.e_pos}")
print(f"\nCommanded acceleration: {ctrl_state.a_cmd.round(3)}")
print(f"\nDesired rotation matrix (should pitch forward):")
print(ctrl_state.R_des.round(4))
print(f"\nControl outputs:")
print(f"  Thrust: {control.thrust_N:.3f} N")
print(f"  Moments: {control.moments_Nm.round(4)} N·m")
print(f"\nNonzero pitch moment to tilt toward target.")

In [ ]:
# Examine controller gains
print("=== Controller Gains ===")
print(f"\nPosition gains (outer loop):")
print(f"  Kp_pos: {params.kp_pos}")
print(f"  Kd_pos: {params.kd_pos}")
print(f"\nAttitude gains (inner loop):")
print(f"  Kr_att: {params.kr_att}")
print(f"  Kw_rate: {params.kw_rate}")
print(f"\nActuator limits:")
print(f"  Thrust: [{params.T_min}, {params.T_max}] N")
print(f"  Moments: ±{params.tau_max} N·m")

## 5. Closed-Loop Simulation

Let's simulate the full closed-loop system with the SE(3) controller tracking various trajectories.

In [ ]:
def simulate_tracking(state_init, trajectory_fn, params, T_sim=5.0, dt=0.002):
    """
    Simulate closed-loop tracking.
    
    Args:
        state_init: Initial state
        trajectory_fn: Function(t) -> TrajPoint
        params: Controller/simulation parameters
        T_sim: Simulation duration [s]
        dt: Timestep [s]
    
    Returns:
        Dictionary of logged data
    """
    n_steps = int(T_sim / dt)
    
    # Allocate storage
    log = {
        't': np.zeros(n_steps),
        'p': np.zeros((n_steps, 3)),
        'v': np.zeros((n_steps, 3)),
        'euler': np.zeros((n_steps, 3)),
        'w': np.zeros((n_steps, 3)),
        'p_des': np.zeros((n_steps, 3)),
        'v_des': np.zeros((n_steps, 3)),
        'thrust': np.zeros(n_steps),
        'moments': np.zeros((n_steps, 3)),
        'e_pos': np.zeros((n_steps, 3)),
        'e_att': np.zeros((n_steps, 3)),
    }
    
    state = state_init.copy()
    
    for i in range(n_steps):
        t = i * dt
        traj = trajectory_fn(t)
        
        # Controller
        control, ctrl_state = se3_controller(state, traj, params)
        
        # Log
        log['t'][i] = t
        log['p'][i] = state.p
        log['v'][i] = state.v
        log['euler'][i] = np.degrees(quat_to_euler(state.q))
        log['w'][i] = state.w_body
        log['p_des'][i] = traj.p
        log['v_des'][i] = traj.v
        log['thrust'][i] = control.thrust_N
        log['moments'][i] = control.moments_Nm
        log['e_pos'][i] = ctrl_state.e_pos
        log['e_att'][i] = ctrl_state.e_att
        
        # Dynamics
        state = step_rk4(state, control, params, dt)
    
    return log

In [ ]:
# Simulation 1: Step response in altitude
params = Params()

# Start at z=0, step to z=1
state_init = State.zeros()

def altitude_step(t):
    return TrajPoint.hover(position=np.array([0.0, 0.0, 1.0]))

log = simulate_tracking(state_init, altitude_step, params, T_sim=4.0)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Position
axes[0, 0].plot(log['t'], log['p'][:, 2], 'b-', label='Actual Z', linewidth=2)
axes[0, 0].plot(log['t'], log['p_des'][:, 2], 'r--', label='Desired Z', linewidth=1.5)
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Altitude [m]')
axes[0, 0].set_title('Altitude Step Response')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Velocity
axes[0, 1].plot(log['t'], log['v'][:, 2], 'b-', linewidth=2)
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Vertical Velocity [m/s]')
axes[0, 1].set_title('Vertical Velocity')
axes[0, 1].grid(True)

# Thrust
axes[1, 0].plot(log['t'], log['thrust'], 'b-', linewidth=2)
axes[1, 0].axhline(y=params.hover_thrust, color='r', linestyle='--', label=f'Hover ({params.hover_thrust:.2f} N)')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Thrust [N]')
axes[1, 0].set_title('Thrust Command')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Position error
axes[1, 1].plot(log['t'], log['e_pos'][:, 2], 'b-', linewidth=2)
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Z Error [m]')
axes[1, 1].set_title('Position Error')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"Final position: {log['p'][-1].round(4)}")
print(f"Final error: {np.linalg.norm(log['e_pos'][-1]):.4f} m")

In [ ]:
# Simulation 2: Lateral step (horizontal movement)
state_init = State.zeros()
state_init.p = np.array([0.0, 0.0, 1.0])  # Start at hover

def lateral_step(t):
    return TrajPoint.hover(position=np.array([1.0, 0.0, 1.0]))  # Move to +X

log = simulate_tracking(state_init, lateral_step, params, T_sim=4.0)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# XY trajectory
axes[0, 0].plot(log['p'][:, 0], log['p'][:, 1], 'b-', linewidth=2, label='Actual')
axes[0, 0].scatter(0, 0, c='g', s=100, zorder=5, label='Start')
axes[0, 0].scatter(1, 0, c='r', s=100, marker='x', zorder=5, label='Target')
axes[0, 0].set_xlabel('X [m]')
axes[0, 0].set_ylabel('Y [m]')
axes[0, 0].set_title('XY Trajectory')
axes[0, 0].legend()
axes[0, 0].axis('equal')
axes[0, 0].grid(True)

# X position
axes[0, 1].plot(log['t'], log['p'][:, 0], 'b-', label='Actual X', linewidth=2)
axes[0, 1].plot(log['t'], log['p_des'][:, 0], 'r--', label='Desired X', linewidth=1.5)
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('X Position [m]')
axes[0, 1].set_title('X Position')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Euler angles
axes[1, 0].plot(log['t'], log['euler'][:, 0], 'r-', label='Roll', linewidth=2)
axes[1, 0].plot(log['t'], log['euler'][:, 1], 'g-', label='Pitch', linewidth=2)
axes[1, 0].plot(log['t'], log['euler'][:, 2], 'b-', label='Yaw', linewidth=2)
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Angle [deg]')
axes[1, 0].set_title('Euler Angles')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Moments
axes[1, 1].plot(log['t'], log['moments'][:, 0], 'r-', label='Roll τ', linewidth=2)
axes[1, 1].plot(log['t'], log['moments'][:, 1], 'g-', label='Pitch τ', linewidth=2)
axes[1, 1].plot(log['t'], log['moments'][:, 2], 'b-', label='Yaw τ', linewidth=2)
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Moment [N·m]')
axes[1, 1].set_title('Body Moments')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"Note: Pitch tilts to accelerate forward, then tilts back to decelerate.")

In [ ]:
# Simulation 3: Circular trajectory
state_init = State.zeros()
state_init.p = np.array([1.0, 0.0, 1.0])  # Start on the circle

radius = 1.0
omega_circle = 1.0  # rad/s

def circle_trajectory(t):
    angle = omega_circle * t
    # Position
    p = np.array([radius * np.cos(angle), radius * np.sin(angle), 1.0])
    # Velocity
    v = np.array([-radius * omega_circle * np.sin(angle), 
                   radius * omega_circle * np.cos(angle), 0.0])
    # Acceleration (centripetal)
    a = np.array([-radius * omega_circle**2 * np.cos(angle),
                  -radius * omega_circle**2 * np.sin(angle), 0.0])
    return TrajPoint(p=p, v=v, a=a, yaw=0.0, yaw_rate=0.0)

log = simulate_tracking(state_init, circle_trajectory, params, T_sim=8.0)

fig = plt.figure(figsize=(14, 10))

# 3D trajectory
ax3d = fig.add_subplot(2, 2, 1, projection='3d')
ax3d.plot(log['p'][:, 0], log['p'][:, 1], log['p'][:, 2], 'b-', linewidth=2, label='Actual')
ax3d.plot(log['p_des'][:, 0], log['p_des'][:, 1], log['p_des'][:, 2], 'r--', linewidth=1, label='Desired')
ax3d.set_xlabel('X [m]')
ax3d.set_ylabel('Y [m]')
ax3d.set_zlabel('Z [m]')
ax3d.set_title('3D Trajectory')
ax3d.legend()

# XY view
ax1 = fig.add_subplot(2, 2, 2)
ax1.plot(log['p'][:, 0], log['p'][:, 1], 'b-', linewidth=2, label='Actual')
ax1.plot(log['p_des'][:, 0], log['p_des'][:, 1], 'r--', linewidth=1, label='Desired')
ax1.scatter(log['p'][0, 0], log['p'][0, 1], c='g', s=100, zorder=5, label='Start')
ax1.set_xlabel('X [m]')
ax1.set_ylabel('Y [m]')
ax1.set_title('XY Plane (Top View)')
ax1.axis('equal')
ax1.legend()
ax1.grid(True)

# Position error
ax2 = fig.add_subplot(2, 2, 3)
error_norm = np.linalg.norm(log['e_pos'], axis=1)
ax2.plot(log['t'], error_norm, 'b-', linewidth=2)
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Position Error [m]')
ax2.set_title('Tracking Error')
ax2.grid(True)

# Euler angles
ax3 = fig.add_subplot(2, 2, 4)
ax3.plot(log['t'], log['euler'][:, 0], 'r-', label='Roll', linewidth=2)
ax3.plot(log['t'], log['euler'][:, 1], 'g-', label='Pitch', linewidth=2)
ax3.plot(log['t'], log['euler'][:, 2], 'b-', label='Yaw', linewidth=2)
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Angle [deg]')
ax3.set_title('Euler Angles')
ax3.legend()
ax3.grid(True)

plt.tight_layout()
plt.show()

print(f"Mean tracking error: {np.mean(error_norm[500:]):.4f} m")  # Skip transient
print(f"Max tracking error: {np.max(error_norm):.4f} m")

In [ ]:
# Simulation 4: Yaw tracking with constant position
state_init = State.zeros()
state_init.p = np.array([0.0, 0.0, 1.0])

yaw_rate = np.pi / 4  # 45 deg/s

def yaw_trajectory(t):
    yaw = yaw_rate * t
    return TrajPoint(
        p=np.array([0.0, 0.0, 1.0]),
        v=np.zeros(3),
        a=np.zeros(3),
        yaw=yaw,
        yaw_rate=yaw_rate,
    )

log = simulate_tracking(state_init, yaw_trajectory, params, T_sim=10.0)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Yaw angle
axes[0, 0].plot(log['t'], log['euler'][:, 2], 'b-', label='Actual', linewidth=2)
desired_yaw = np.degrees(yaw_rate * log['t'])
# Wrap to [-180, 180]
desired_yaw_wrapped = np.mod(desired_yaw + 180, 360) - 180
axes[0, 0].plot(log['t'], desired_yaw_wrapped, 'r--', label='Desired', linewidth=1.5)
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Yaw [deg]')
axes[0, 0].set_title('Yaw Tracking')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Angular velocity
axes[0, 1].plot(log['t'], log['w'][:, 2], 'b-', linewidth=2)
axes[0, 1].axhline(y=yaw_rate, color='r', linestyle='--', label=f'Desired ({np.degrees(yaw_rate):.1f}°/s)')
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Yaw Rate [rad/s]')
axes[0, 1].set_title('Yaw Rate')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Position (should stay constant)
axes[1, 0].plot(log['t'], log['p'][:, 0], 'r-', label='X', linewidth=2)
axes[1, 0].plot(log['t'], log['p'][:, 1], 'g-', label='Y', linewidth=2)
axes[1, 0].plot(log['t'], log['p'][:, 2], 'b-', label='Z', linewidth=2)
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Position [m]')
axes[1, 0].set_title('Position (should be constant)')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Yaw moment
axes[1, 1].plot(log['t'], log['moments'][:, 2], 'b-', linewidth=2)
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Yaw Moment [N·m]')
axes[1, 1].set_title('Yaw Torque')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 6. Controller Gain Effects

Let's explore how different gains affect the controller performance.

In [ ]:
# Compare different position gains
from quad.params import aggressive_params

state_init = State.zeros()
def step_traj(t):
    return TrajPoint.hover(position=np.array([0.0, 0.0, 1.0]))

# Default gains
params_default = Params()
log_default = simulate_tracking(state_init.copy(), step_traj, params_default, T_sim=3.0)

# Aggressive gains
params_aggressive = aggressive_params()
log_aggressive = simulate_tracking(state_init.copy(), step_traj, params_aggressive, T_sim=3.0)

# Low gains (slower)
params_slow = Params(
    kp_pos=np.array([3.0, 3.0, 4.0]),
    kd_pos=np.array([2.0, 2.0, 2.5]),
)
log_slow = simulate_tracking(state_init.copy(), step_traj, params_slow, T_sim=3.0)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(log_default['t'], log_default['p'][:, 2], 'b-', label='Default', linewidth=2)
plt.plot(log_aggressive['t'], log_aggressive['p'][:, 2], 'r-', label='Aggressive', linewidth=2)
plt.plot(log_slow['t'], log_slow['p'][:, 2], 'g-', label='Slow', linewidth=2)
plt.axhline(y=1.0, color='k', linestyle='--', alpha=0.5)
plt.xlabel('Time [s]')
plt.ylabel('Altitude [m]')
plt.title('Altitude Response - Different Gains')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(log_default['t'], log_default['thrust'], 'b-', label='Default', linewidth=2)
plt.plot(log_aggressive['t'], log_aggressive['thrust'], 'r-', label='Aggressive', linewidth=2)
plt.plot(log_slow['t'], log_slow['thrust'], 'g-', label='Slow', linewidth=2)
plt.axhline(y=params_default.hover_thrust, color='k', linestyle='--', alpha=0.5, label='Hover')
plt.xlabel('Time [s]')
plt.ylabel('Thrust [N]')
plt.title('Thrust - Different Gains')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Default gains: Kp = {params_default.kp_pos[2]}, Kd = {params_default.kd_pos[2]}")
print(f"Aggressive gains: Kp = {params_aggressive.kp_pos[2]}, Kd = {params_aggressive.kd_pos[2]}")
print(f"Slow gains: Kp = {params_slow.kp_pos[2]}, Kd = {params_slow.kd_pos[2]}")

## Summary

The `controller_se3.py` module provides:

1. **`compute_desired_rotation(a_cmd, yaw)`**: Constructs desired rotation from thrust direction and yaw
2. **`attitude_error(R, R_des)`**: Geometric attitude error on SO(3)
3. **`angular_rate_error(w, w_des, R, R_des)`**: Angular rate error with frame transformation
4. **`se3_controller(state, traj, params)`**: Full SE(3) tracking controller

### Key Properties

- **Cascaded structure**: Position loop commands attitude, attitude loop commands moments
- **Geometric formulation**: No singularities (unlike Euler angle controllers)
- **Gyroscopic compensation**: Feedforward term for angular momentum
- **Saturation**: Thrust and moments are clipped to physical limits

### Tuning Guidelines

- **Position gains (Kp, Kd)**: Higher = faster response, but may cause overshoot
- **Attitude gains (Kr, Kw)**: Inner loop, should be faster than outer loop
- **Critically damped**: Kd ≈ 2√(Kp) for smooth response without overshoot

### Next Steps

- See `trajectory.py` for generating smooth reference trajectories
- Run `main.py` for complete simulation examples
- Explore `plots.py` for visualization utilities